# Week 1 Lab: next generation sequencing and mutation hunting

## Part 1
**Due: Monday 4/8/19**
Skills covered: basic UNIX navigation, intro to NGS, sequence alignment and variant calling

TODO: description of this week's lab
TODO: this week most commands given to you. as we go on you'll be provided less info

## 1. Inspect raw data manually

Before running any analysis, it's a good practice to look manually at the data to make sure all the files are what you think they are. First, you should look at the sequence files to verify the format is correct. You don’t want to open the entire files, you will just do a quick inspection of the first few reads. Use the `head` command to look at the first 20 lines in the first fastq file for the child genome (`NA12878_child_1.fq`)
your file (replace [filename.format] with the actual file name).

```shell
head -n 20 [file.format]
```

Take a look at each file. Each read has 4 lines of information, and then the next read starts on the
following line. The first line starts with the "@" symbol, and contains identifiers and information about
this read. The next line contains the actual sequence, then on line three there is a "+", which may
sometimes have the identifier and info repeated. 

Line 4 contains the quality string, where ASCII characters encode the quality score for each base.
The quality score ranges from 0 to about 40; the higher the number, the greater the confidence of the
base call. To get the actual quality score, you need to figure out the value of the symbol, then subtract
33 (this is called "Sanger" scaling type, because it's the same scaling that people use with traditional
sanger sequencing). With some older Illumina data (pre version 1.8), you subtract other numbers, like
64, for more info, see: 

http://drive5.com/usearch/manual/quality_score.html

Also take a look at the second file, containing the second read in each read pair. Notice the reads are in the same order in each file. e.g. the first reads in `NA12878_child_1.fq` and in `NA12878_child_2.fq` are paired. They have identical read identifiers, with the exception of the suffix `<read id>/1` vs. `<read id>/2`.

<font color="red" size="3">**Question 1 (0.5 pts)**: Look at the beginning of the `NA12878_child_1.fq` file, and find the third read. What is the read identifier? What is the quality symbol for the read's first base? Look up the value associated with this ASCII symbol online. What is the probability that this base is an error?</font>

<font color="blue" size="3">**Answer 1**</font>

Read ID: 

ASCII symbol:

P(error):



Now let's take a look at the human reference genome, `hg19.fa`, and introduce a new UNIX command. We can use the `cat` command to print the contents of a file to the screen. 

The reference genome is a pretty big file (3+ GB) so we don't want to look at the whole thing at once. Instead, we'll use a UNIX trick of "piping" (using the `|` symbol) which allows us to take the output of one command and use it as input to the next:

```
cat hg19.fa | head -n 10
```

This command first ran `cat` to print the file to the screen (which we call "standard output"). Then the command of `head` used the output of `cat` as its input rather than specifying a file directly as we did above. We refer to this as "standard input".

<blockquote>
**UNIX TIP**: If you accidently open (e.g. cat) a huge file and just see characters flying down your screen, matrix style, or if you think a process you are running may be stuck, you can press control+c to cancel the last command.
</blockquote>

<font color="red" size="3">**Question 2 (0.5 pts)**: What is the name of the first chromosome listed in the reference genome? The top of the file has a bunch of "N"'s. Why do you think that is?</font>

<font color="blue" size="3">**Answer 2**</font>

We can string together a bunch of commands which can be pretty flexible. e.g. if you want look further down the file at lines 9990 through 10,000 you can do:

```
cat hg19.fa | head -n 10000 | tail -n 10
```

which prints the reference genome, takes the top (`head`) 10,000 lines, then the last (`tail`) 10 lines of that.

<font color="red" size="3">**Question 3 (0.5 pts)**: Write a one-line UNIX command to print out the read ID of the 100th read (and nothing else) from the file `NA12878_1.fq. (Hint: remember each read takes up 4 lines)</font>

<font color="blue" size="3">**Answer 3**</font>

We'll introduce one more helpful UNIX command before moving on. The tool `wc` includes a helpful option to count the number of lines in a file:

```
wc -l [filename]
```

<blockquote>
If you want more details about a UNIX tool, you can try looking at the manual using `man`, e.g. `man wc -l`
</blockquote>

<font color="red" size="3">**Question 4 (0.25 pts)**: Using `wc -l`, determine how many read pairs we have for each of the three samples (child, mother, and father)</font>

<font color="blue" size="3">**Answer 4**</font>

## 2.  Inspect raw sequencing data with fastqc

`cd` into your home directory. Make a directory that will contain all of the data for week1, use `ls` to confirm its there, and `cd` into it.

```
mkdir week1
cd week1
```

This is where you will be working for the rest of the week.

OK, now you are ready to run `fastqc`, a simple fastq statistics analysis program. First, make sure the program is working and properly installed by typing the command below. You should see the manual page. Please ask a TA if this is not working.

```
fastqc -h
```

Run the program fastqc on the fastq files. You will have to specify the path to the `public/week1` folder, since you should leave the files there. You also have to tell `fastqc` to `-o` output the files to the current directory, which you indicate with a `.` The command to do all this is below, but you have to specify the full root path to each fastq file.

```
fastqc -o . NA12878_child_1.fq NA12878_child_2.fq NA12891_father_1.fq NA12891_father_2.fq NA12892_mother_1.fq NA12892_mother_2.fq
```

Check with `ls` that this generated some files. The html files contain the whole report. To look at them, you will need to download them to your local computer.

Open a new terminal window and use pwd and ls to find the path to the desktop. Then use "secure copy" with the command below, using the full rooted path to the html file and your local desktop. If you’re not sure of the path, just pwd while you're in the relevant directory. Copy all htmo files.

```
scp [username]@ieng6.ucsd.edu:[path to your home week 1 directory]/*.html [path to desktop]
```

You will be prompted for your cs185 password. Once the files appear on your desktop, open them and take a look. Do the basic statistics match what you calculated for the number of reads? On the left, you’ll see a navigation window with green (normal), yellow (slightly abnormal), and red (very unusual) circles for several kinds of data analysis. If you have any red circles, record them in your notebook, and read the fastqc documentation on the analysis modules to try to learn what they mean.

http://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/

Our sequencing datasets are high quality and nothing should be flagged in red, indicating we're ready to move on to sequence alignment.

<font color="red" size="3">**Question 5 (0.5 pts)**: Look at the "Per base sequence quality" section of the `fastqc` output for one of the files. What happens to the quality scores as you get toward the end of the read? Why?</font>

<font color="blue" size="3">**Answer 5**</font>

## 3. Aligning sequences to the reference genome

## 4. Meet SAM, inspect the alignment

sam/bam format, tview

## 5. Inspecting the template length distribution

TODO plotting

***That's it for this session! This was just a brief introduction to the data structure from illumina
sequencing runs and downstream analyses. Next time, we'll write some of our own code for variant calling and compare our results to state of the art tools.***